<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.2: Generators: Collections
**Prev: [Generators: Parameters](3.1_parameters.ipynb)**<br>
**Next: [Interlude: Chisel Standard Library](3.2_interlude.ipynb)**

## Motivation
Generators 经常需要处理可变数量的对象，无论是 IOs、模块还是测试向量。集合（collections）是处理这种情况的重要构建块。本模块将介绍 Scala 集合以及如何在 Chisel Generators 中使用它们。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Note we add a new import here because `mutable.ArrayBuffer` lives in `scala.collections`.

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import scala.collection._

---
# Generators and Collections<a name="generators-and-collections"></a>
在本节中，我们将重点介绍*Generators*的概念以及使用 Scala 集合作为实现它们的工具。我们将不再将 Chisel 代码视为电路的*实例*，即特定电路的描述，而是将其视为电路的生成器。

我们将从之前练习中的 FIR 滤波器开始。

In [ ]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}


这个电路是生成器的一个简单例子，因为它可以生成具有不同系数的这个 4-tap 滤波器的不同版本。但如果我们希望电路有更多的 tap 该怎么办？我们将分几个步骤来实现这一点。

- 构建一个软件的 *Golden Model* 用于 tap 可配置的 FIR。
- 重新设计我们的测试以使用这个模型，并确认它工作正常。
- 重构我们的 My4ElementFir 以允许可配置的 tap 数量。
- 使用我们的新测试平台测试新电路。

<span style="color:blue">**示例：FIR Golden Model**</span><br><a name="fir-golden-model"></a>
下面是一个 FIR 电路的 Scala 软件实现。

In [ ]:
/**
  * A naive implementation of an FIR filter with an arbitrary number of taps.
  */
class ScalaFirFilter(taps: Seq[Int]) {
  var pseudoRegisters = List.fill(taps.length)(0)

  def poke(value: Int): Int = {
    // 将新的输入值 value 插入到 pseudoRegisters 的开头，同时丢弃列表末尾的一个元素
    // :: 是列表连接操作符，将 value 添加到现有列表的前面。
    pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)
    var accumulator = 0
    for(i <- taps.indices) {
      accumulator += taps(i) * pseudoRegisters(i)
    }
    accumulator
  }
}

### Seq
注意，`taps` 变成了一个 `Seq[Int]`，这意味着在构建类时，类的用户可以传递任意长度的 `Int` 序列。
### Registers
使用 `var pseudoRegisters = List.fill(taps.length)(0)`，我们创建了一个 `List`，该 `List` 将保存前几周期的值。选择 `List` 是因为它在语法上添加一个元素到头部并移除最后一个元素非常简单。几乎任何 Scala 集合家族的成员都可以使用。我们还将此列表初始化为全零。
### Poke
我们的类添加了一个 poke 函数/方法，该函数/方法模拟将新输入放入滤波器并循环时钟。
### Updating the registers
这行 `pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)` 首先使用列表的 `take` 方法保留列表中除最后一个元素之外的所有元素，然后使用 `::` 列表连接运算符将 `value` 添加到列表缩减版的头部。
### Computing the output
一个带有累加器的简单 for 循环将列表中的每个元素乘以其对应的 tap 系数并求和。只有 `accumulator` 的那一行返回该值作为函数结果。
## Adapting our previous test for testing our golden model
我们现在将使用之前的工作来确认我们的 golden model 是否正常工作。一些编辑魔法将我们以前的测试平台变成了……

In [ ]:
val filter = new ScalaFirFilter(Seq(1, 1, 1, 1))

var out = 0

out = filter.poke(1)
println(s"out = $out")
assert(out == 1)  // 1, 0, 0, 0

out = filter.poke(4)
assert(out == 5)  // 4, 1, 0, 0
println(s"out = $out")

out = filter.poke(3)
assert(out == 8)  // 3, 4, 1, 0
println(s"out = $out")

out = filter.poke(2)
assert(out == 10)  // 2, 3, 4, 1
println(s"out = $out")

out = filter.poke(7)
assert(out == 16)  // 7, 2, 3, 4
println(s"out = $out")

out = filter.poke(0)
assert(out == 12)  // 0, 7, 2, 3
println(s"out = $out")

执行前面的代码块可以证明，我们的软件模型返回的结果与 My4ElementFir 一致。

## Test circuit using the golden model.<a name="use-golden-model-as-test"></a>
既然我们对我们的 golden model 充满信心，我们就可以重写我们的测试，将电路输出与 golden model 的输出进行比较，而不是使用费力手工制作的示例。以下是快速实现的初步尝试。

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

test(new My4ElementFir(1, 1, 1, 1)) { c =>
    for(i <- 0 until 100) {
        val input = scala.util.Random.nextInt(8)

        val goldenModelResult = goldenModel.poke(input)

        c.io.in.poke(input.U)

        c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

        c.clock.step(1)
    }

}


我们的测试运行了 100 个周期，并在每一步检查硬件和软件这两种不同方法是否同步。

### 需要注意的事项
（即，我们在编写此内容时实际犯的错误。）

1. 将 step 放在正确的位置。软件和硬件执行的方式不同；很容易出错。
2. 此测试较弱，因为它对 IO 和寄存器的大小非常敏感。实现一个在任意数据位宽上观察环绕行为的软件 golden model 可能会很复杂。在这里，我们只是确保传入的值是合适的。

<span style="color:blue">**示例：参数化 FIR 生成器**</span><br><a name="fir-golden-model"></a> 
下面我们创建了一个新的 Filter 类 `MyManyElementsFilter`，它接受一个用于 taps 的常量 `Seq`。这个列表可以包含任意数量的元素。为了更好地测量，还添加了一个 `bitWidth`，允许我们控制电路可以处理的数字大小。针对可变长度，我们不得不重构寄存器的创建方式及其连接方式。下面的方法使用了可用库集合函数的一个简单子集。后续章节将展示如何更简洁地表达行为，并使发生的事情更清晰。

In [ ]:
class MyManyElementFir(consts: Seq[Int], bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan.last
}

#### 我们是如何做到的
从第 7 行、第 13 行和第 18 行开始，有三个并行部分。
我们使用了一种 Scala 集合类型，称为 `ArrayBuffer`。
`ArrayBuffer` 允许你使用 `+=` 操作符追加元素（也可以插入和删除元素，但我们不需要这些）。
首先，我们创建一个元素为 `UInt` 的 ArrayBuffer `regs`。
然后遍历 taps，将输入作为第一个元素添加，然后使用 `RegNext` 创建寄存器，将寄存器的输入连接到前一个元素（`regs(i-1)`），并将其初始化为无符号零（`0.U`）。
这些寄存器将根据需要保存输入的先前值。

接下来，我们创建另一个 `UInt` 类型的 ArrayBuffer `muls`。
`muls` 的每个元素将是一个节点，其第 i 个元素是 `regs(i)` 和 `const(i)` 的乘积。

注意 `scan.last` 方法的使用。
它获取集合的最后一个元素，是在构建 `regs` 时使用 `regs(i - 1)` 的更优雅的替代方法。

### 它的行为与 `My4ElementFir` 一样吗？
对新版本的一个很好的初步测试是看它是否能通过我们刚刚应用于 `My4ElementFir` 的测试。我们创建一个 `MyManyElementFir` 的实例，并通过它运行更多的数据。

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

test(new MyManyElementFir(Seq(1, 1, 1, 1), 8)) { c =>
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      c.io.in.poke(input.U)

      c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

      c.clock.step(1)
    }
}

### 现在我们来测试一组不同大小的 FIR 滤波器
我们创建一些辅助函数：`r` 用于获取一个随机数；`runOneTest` 用于为特定的一组 taps 创建一个 golden model 和硬件仿真滤波器，然后通过滤波器运行至少两倍于 taps 数量的数据。

In [ ]:
/** a convenience method to get a random integer
  */
def r(): Int = {
  scala.util.Random.nextInt(1024)
}

/**
  * run a test comparing software and hardware filters
  * run for at least twice as many samples as taps
  */
def runOneTest(taps: Seq[Int]) {
    val goldenModel = new ScalaFirFilter(taps)

    test(new MyManyElementFir(taps, 32)) { c =>
        for(i <- 0 until 2 * taps.length) {
            val input = r()

            val goldenModelResult = goldenModel.poke(input)

            c.io.in.poke(input.U)

            c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

            c.clock.step(1)
        }
    }
}

for(tapSize <- 2 until 100 by 10) {
    val taps = Seq.fill(tapSize)(r())  // create a sequence of random coefficients

    runOneTest(taps)
}

### Just for fun, let's make a bigger one
以下代码将在一个 500 tap 的 FIR 滤波器上运行单个测试。运行可能需要一分钟左右。
（提示：当执行完成时，注意工具栏上的 Scala ● 变为 Scala ○。）

In [ ]:
runOneTest(Seq.fill(500)(r()))

# Hardware Collections

<span style="color:blue">**Example: Add run-time configurable taps to our FIR**</span><br>
以下代码向我们的 FIR 生成器的 IO 添加了一个额外的 `consts` 向量，允许在电路生成后从外部更改系数。这是通过 Chisel 集合类型 `Vec` 完成的。
`Vec` 支持许多 Scala 集合方法，但它只能包含 Chisel 硬件元素。
`Vec` 应仅在普通 Scala 集合不起作用的情况下使用。
基本上这是在以下两种情况下之一：
1. 在 Bundle 中需要元素的集合（collection），通常是将用作 IO 的 Bundle。
2. 你需要通过作为硬件一部分的索引访问集合（想想寄存器文件）。（译者注：Vec 主要在需要通过硬件索引访问元素的情况下使用，比如寄存器文件。如果不需要动态索引，而只是简单地按顺序处理元素，可以使用 Seq 或其他 Scala 集合）

In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = RegInit(VecInit(Seq.fill(length - 1)(0.U(8.W))))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

test(new MyManyDynamicElementVecFir(4)) { c =>
    c.io.consts(0).poke(1.U)
    c.io.consts(1).poke(1.U)
    c.io.consts(2).poke(1.U)
    c.io.consts(3).poke(1.U)
    for(i <- 0 until 100) {
        val input = scala.util.Random.nextInt(8)

        val goldenModelResult = goldenModel.poke(input)

        c.io.in.poke(input.U)

        c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

        c.clock.step(1)
    }
}


<span style="color:red">**Exercise: 32-bit RISC-V Processor**</span><br><a name="fir-golden-model"></a>

[寄存器文件](https://en.wikipedia.org/wiki/Register_file) 是构建处理器的重要组成部分。寄存器文件是一个寄存器数组，可以通过多个读或写端口进行读取或写入。每个端口由一个地址和数据字段组成。

[RISC-V 指令集架构](https://riscv.org/specifications/) 定义了几种变体，其中最简单的称为 RV32I。RV32I 有一个大小为 32 的 32 位寄存器数组。
**从索引为 0（第一个寄存器）的寄存器读取时，该寄存器始终为零，无论写入什么内容**（方便使用零）。

为 RV32I 实现一个具有单写端口和参数化读端口数量的寄存器文件。当 `wen`（写使能）被置位时才执行写操作。

In [ ]:
class RegisterFile(readPorts: Int) extends Module {
    require(readPorts >= 0)
    val io = IO(new Bundle {
        val wen   = Input(Bool())
        val waddr = Input(UInt(5.W))
        val wdata = Input(UInt(32.W))
        val raddr = Input(Vec(readPorts, UInt(5.W)))
        val rdata = Output(Vec(readPorts, UInt(32.W)))
    })
    
    // A Register of a vector of UInts
    val reg = RegInit(VecInit(Seq.fill(32)(0.U(32.W))))
    
    ???

    
}

In [ ]:
test(new RegisterFile(2) ) { c =>
  def readExpect(addr: Int, value: Int, port: Int = 0): Unit = {
    c.io.raddr(port).poke(addr.U)
    c.io.rdata(port).expect(value.U)
  }
  def write(addr: Int, value: Int): Unit = {
    c.io.wen.poke(true.B)
    c.io.wdata.poke(value.U)
    c.io.waddr.poke(addr.U)
    c.clock.step(1)
    c.io.wen.poke(false.B)
  }
  // everything should be 0 on init
  for (i <- 0 until 32) {
    readExpect(i, 0, port = 0)
    readExpect(i, 0, port = 1)
  }

  // write 5 * addr + 3
  for (i <- 0 until 32) {
    write(i, 5 * i + 3)
  }

  // check that the writes worked
  for (i <- 0 until 32) {
    readExpect(i, if (i == 0) 0 else 5 * i + 3, port = i % 2)
  }
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
    when (io.wen) {
        reg(io.waddr) := io.wdata
    }
    for (i &lt;- 0 until readPorts) {
        when (io.raddr(i) === 0.U) {
            io.rdata(i) := 0.U
        } .otherwise {
            io.rdata(i) := reg(io.raddr(i))
        }
    }

</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)